In [9]:
# Import the pygame module
# Import random for random numbers
import random

import pygame

# Import pygame.locals for easier access to key coordinates
# Updated to conform to flake8 and black standards
from pygame.locals import (
    K_DOWN,
    K_ESCAPE,
    K_LEFT,
    K_RIGHT,
    K_UP,
    KEYDOWN,
    QUIT,
)

# Define constants for the screen width and height
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600

pygame.font.init()
my_font = pygame.font.SysFont('Comic Sans MS', 30)

# Define the Player object extending pygame.sprite.Sprite
# The surface we draw on the screen is now a property of 'player'
class Player(pygame.sprite.Sprite):
    def __init__(self):
        super(Player, self).__init__()
        self.surf = pygame.Surface((75, 25))
        self.surf.fill((0, 255, 0))
        self.rect = self.surf.get_rect()
        self.score = 0
        self.health = 1000
        self.level = 1

    # Move the sprite based on keypresses
    def update(self, pressed_keys):
        self.score += 1
        if(self.score > self.level * 1000):
            self.level += 1

        self.rect.move_ip(0, 1)
        
        self.score_surface = my_font.render("Score: " + str(self.score), False, (8, 100, 240))
        self.health_surface = my_font.render("Health: " + str(self.health), False, (8, 100, 240))  
        self.level_surface = my_font.render("Level: " + str(self.level), False, (8 , 100, 240))
        
        if pressed_keys[K_UP]:
            self.rect.move_ip(0, -5)
        if pressed_keys[K_DOWN]:
            self.rect.move_ip(0, 5)
        if pressed_keys[K_LEFT]:
            self.rect.move_ip(-5, 0)
        if pressed_keys[K_RIGHT]:
            self.rect.move_ip(5, 0)

        # Keep player on the screen
        if self.rect.left < 0:
            self.rect.left = 0
        elif self.rect.right > SCREEN_WIDTH:
            self.rect.right = SCREEN_WIDTH
        if self.rect.top <= 0:
            self.rect.top = 0
        elif self.rect.bottom >= SCREEN_HEIGHT:
            self.rect.bottom = SCREEN_HEIGHT

    def reduce_health(self):
        self.health -= 1
        if self.health == 0:
            self.kill()
            return False
        return True

    def increase_health(self):
        if self.health < 1000:
            self.health += 1

# Define the enemy object extending pygame.sprite.Sprite
# The surface we draw on the screen is now a property of 'enemy'
class Enemy(pygame.sprite.Sprite):
    def __init__(self, difficulty):
        super(Enemy, self).__init__()
        # size
        self.surf = pygame.Surface((30, 30))
        self.surf.fill((0,100,255))
        # location
        self.rect = self.surf.get_rect(
            center=(
                SCREEN_WIDTH + 30,
                random.randint(0, SCREEN_HEIGHT),
            )
        )
        self.speed = random.randint(1 + difficulty, 2 + difficulty)

    # Move the sprite based on speed
    # Remove it when it passes the left edge of the screen
    def update(self):
        C = random.randint(-2, 2)
        self.rect.move_ip(-self.speed, C)
        if self.rect.right < 0:
            self.kill()

class Health(pygame.sprite.Sprite):
    def __init__(self):
        super(Health, self).__init__()
        self.surf = pygame.Surface((30, 35))
        self.surf.fill((200,0,0))
        # location
        self.rect = self.surf.get_rect(
            center=(
                SCREEN_WIDTH + 30,
                random.randint(0, SCREEN_HEIGHT),
            )
        )
        self.speed = 3       
        
    def update(self):
        self.rect.move_ip(-self.speed, 0)
        if self.rect.right < 0:
            self.kill()

# Initialize pygame
pygame.init()

# Create the screen object
# The size is determined by the constant SCREEN_WIDTH and SCREEN_HEIGHT
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

# Create a custom event for adding a new enemy.
ADDENEMY = pygame.USEREVENT
pygame.time.set_timer(ADDENEMY, 500)

# Create a custom event for adding health
ADDHEALTH = pygame.USEREVENT + 1
pygame.time.set_timer(ADDHEALTH, 2000)

# Create our 'player'
player = Player()

# Create groups to hold enemy sprites, and every sprite
# - enemies is used for collision detection and position updates
# - all_sprites is used for rendering
enemies = pygame.sprite.Group()
health = pygame.sprite.Group()

all_sprites = pygame.sprite.Group()
all_sprites.add(player)

# Variable to keep our main loop running
running = True

# Our main loop
while running:
    # Look at every event in the queue
    for event in pygame.event.get():
        # Did the user hit a key?
        if event.type == KEYDOWN:
            # Was it the Escape key? If so, stop the loop
            if event.key == K_ESCAPE:
                running = False

        # Did the user click the window close button? If so, stop the loop
        elif event.type == QUIT:
            running = False

        # Should we add a new enemy?
        elif event.type == ADDENEMY:
            # Create the new enemy, and add it to our sprite groups
            new_enemy = Enemy(player.level)
            enemies.add(new_enemy)
            all_sprites.add(new_enemy)

        elif event.type == ADDHEALTH:
            new_health = Health()
            health.add(new_health)
            all_sprites.add(new_health)

    # Get the set of keys pressed and check for user input
    pressed_keys = pygame.key.get_pressed()
    player.update(pressed_keys)

    # Update the position of our enemies
    enemies.update()
    health.update()
    
    # Fill the screen with black
    screen.fill((0, 0, 0))

    # Draw all our sprites
    for entity in all_sprites:
        screen.blit(entity.surf, entity.rect)

    screen.blit(player.score_surface, (0,0))
    screen.blit(player.health_surface, (0,30))
    screen.blit(player.level_surface, (0,60))
    
    # Check if any enemies have collided with the player
    if pygame.sprite.spritecollideany(player, enemies):
        # If so...
        if player.reduce_health() == False:
            running = False

    if pygame.sprite.spritecollideany(player, health):
        player.increase_health()

    # Flip everything to the display
    pygame.display.flip()

pygame.quit()